A3-Q2

Rollnumber 2023702013



1. [0.5 points] Data preparation. Download a sample of the Open Images Dataset v7, specially catering to ducks https://www.kaggle.com/datasets/haziqasajid5122/yolov8-finetuning-dataset-ducks. We will use this to fine-tune a Yolo v8 nano detector. Unzip the archive and note how the data is stored separately in the training and validation sets.



The dataset download was a fairly straight forward task and the dataset configuration was stored in the config.yaml file. 

2. [1 point] Understanding YOLO object detector. Study the YOLO object detector in detail. Understand and explain (in your own words, no plagiarism!) how single-shot detectors such as YOLO are different from the R-CNN series that we learned in class. There are more than 8 versions of the YOLO series. Pick any 3 and understand and explain (in your words) the differences between them.

3. Hands on with ultralytics. Familiarize yourself with the ultralytics library (https://docs. ultralytics.com/). Learn more about the train and predict functions of the YOLO class.

(i) Create a yolov8n (yolo-v8-nano) model from scratch. How many parameters does this model have? How many convolutional layers does it use?

(ii) Compare the size (# parameters and # conv layers) of this model against yolov8m (medium) model. Hint: See https://docs.ultralytics.com/models/yolov8/.

In [6]:
import wandb

wandb.login()

wandb.init(project="yolov8-duck-detection", entity="sunayana981")


wandb: Currently logged in as: sunayana-samavedam (sunayana981). Use `wandb login --relogin` to force relogin


In [7]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
model_info1 = model.info()
model_m = YOLO('yolov8m.yaml')
model_info2 = model_m.info()

YOLOv8n summary: 225 layers, 3157200 parameters, 3157184 gradients, 8.9 GFLOPs
YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients, 79.3 GFLOPs


In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


4. [2 points] Training YOLO variants.


(i) Create two versions of the training dataset. train1 containing 100 images. train2 containing all 400 images. You may slice the dataset randomly. Create copies of the config.yaml that came with the dataset zip file accordingly (point to the right directories).

In [9]:
import shutil
import random
import yaml
from pathlib import Path

dataset_dir = Path('/content/drive/My Drive/archive')


train_images_dir = dataset_dir / "images" / "train"
train_labels_dir = dataset_dir / "labels" / "train"
val_images_dir = dataset_dir / "images" / "val"
val_labels_dir = dataset_dir / "labels" / "val"

train_images = list(train_images_dir.glob("*.*"))
val_images = list(val_images_dir.glob("*.*"))


random.shuffle(train_images)
base_dir = Path('/content/Data')


def setup_training_directory(base_dir, name, train_images_subset, val_images_subset):

    images_train_dir = base_dir / name / "images" / "train"
    labels_train_dir = base_dir / name / "labels" / "train"
    images_val_dir = base_dir / name / "images" / "val"
    labels_val_dir = base_dir / name / "labels" / "val"

    images_train_dir.mkdir(parents=True, exist_ok=True)
    labels_train_dir.mkdir(parents=True, exist_ok=True)
    images_val_dir.mkdir(parents=True, exist_ok=True)
    labels_val_dir.mkdir(parents=True, exist_ok=True)

    for image in train_images_subset:
        shutil.copy(image, images_train_dir / image.name)
        label_file = train_labels_dir / (image.stem + ".txt")
        shutil.copy(label_file, labels_train_dir / label_file.name)

    for image in val_images_subset:
        shutil.copy(image, images_val_dir / image.name)
        label_file = val_labels_dir / (image.stem + ".txt")
        shutil.copy(label_file, labels_val_dir / label_file.name)

    config = {
        'path': str(base_dir),
        'train': str(name) + '/images/train',
        'val': str(name) + '/images/val',
        'nc': 1,
        'names': ['duck']
    }
    config_path = base_dir / name / 'config.yaml'
    with open(config_path, 'w') as file:
        yaml.safe_dump(config, file)


setup_training_directory(base_dir, "train1", train_images[:100], val_images)
setup_training_directory(base_dir, "train2", train_images[:400], val_images)

print("Directories and config files for train1 and train2 are setup complete!")


Directories and config files for train1 and train2 are setup complete!


(ii) On both training dataset variants above, train three variants of the Yolo v8 models:

 (a) yolov8n initialized from scratch;
 
  (b) yolov8n from pretrained weights; and 
  
  (c) yolov8m from pretrained weights.


In [ ]:
#1. training yolo v8 nano from scratch on train1
model = YOLO('yolov8n.yaml')
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs = 20)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/My Drive/archive/train1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

100%|██████████| 755k/755k [00:00<00:00, 16.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/Data/train1/labels/train... 222 images, 0 backgrounds, 0 corrupt: 100%|██████████| 222/222 [00:01<00:00, 207.72it/s]

train: New cache created: /content/Data/train1/labels/train.cache


...


20 epochs completed in 1.195 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.2MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:12<00:00,  6.47s/it]


                   all         50         52      0.183     0.0769     0.0926     0.0281
Speed: 2.1ms preprocess, 218.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train4


lr/pg0,▂▃▄▆▇▇██▇▇▆▅▅▄▄▃▃▂▂▁
lr/pg1,▂▃▄▆▇▇██▇▇▆▅▅▄▄▃▃▂▂▁
lr/pg2,▂▃▄▆▇▇██▇▇▆▅▅▄▄▃▃▂▂▁
metrics/mAP50(B),▂▁▁▁▁▂▄▅▁▁▂▂▃▂▃▅█▇█▇
metrics/mAP50-95(B),▂▁▁▁▁▂▃▅▁▂▂▂▃▂▂▅▇▆██
metrics/precision(B),▁▁▁▁▁▁▁▁▁▂▁▂▄▂▃▄▇▃▄█
metrics/recall(B),████▅▇▇▅▆▁▂▂▁▁▃▃▁▆▆▁
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▇▇▆▅▅▄▄▄▃▃▂▃▂▂▁▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x796032232140>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
#2. training yolov8 nano from scratch on train2
model = YOLO('yolov8n.yaml')
model.train(data='/content/drive/My Drive/archive/train2/config.yaml', epochs = 20)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/My Drive/archive/train2/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

100%|██████████| 755k/755k [00:00<00:00, 15.7MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 20.6MB/s]


AMP: checks passed ✅


...


20 epochs completed in 0.083 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.48it/s]


                   all         50         52      0.312      0.288      0.263      0.104
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train


lr/pg0,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg1,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▁▁▁▁▁▁▃▂▃▃▅▇▃▅▆▆██▆█
metrics/mAP50-95(B),▁▁▁▁▁▁▂▂▂▂▃▅▂▅▆▆▇▇▆█
metrics/precision(B),▁▁▁▁▁▁▃▂▃▆▄▇▂▅▇███▅▆
metrics/recall(B),██▆▆█▁▃▃▂▁▃▄▂▃▃▄▄▂▃▄
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▇▆▅▅▄▄▄▄▃▃▂▂▂▂▁▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79720d846830>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
#3. using the pretrained weights of yolo nano on train1

model = YOLO("yolov8n.pt")
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs=20)



Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/archive/train1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/Data/train1/labels/train... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 243.35it/s]


train: New cache created: /content/Data/train1/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train1/labels/val... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 92.32it/s]

...


20 epochs completed in 0.022 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.17it/s]


                   all         50         52     0.0034      0.981      0.691      0.503
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/train2


lr/pg0,▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
lr/pg1,▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
lr/pg2,▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
metrics/mAP50(B),▄█▇▅▅▃▆▅▃▁▂▁▅▅▅▅▄▆▆█
metrics/mAP50-95(B),▅█▇▅▅▃▅▄▃▂▁▁▃▄▅▅▅▅▆█
metrics/precision(B),▁▁▁▁▁▆███▅▅▅█▇▆▆▆█▇▁
metrics/recall(B),████▇▃▂▂▁▁▂▂▂▃▃▃▃▃▄█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▃▄▅▃▅▃▆▃▅▅▆▅▃▂▂▂▁▂


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7971f47827d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
#4. using the pretrained weights of yolo nano on train2

model = YOLO("yolov8n.pt")
model.train(data='/content/drive/My Drive/archive/train2/config.yaml', epochs=20)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/archive/train2/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/Data/train2/labels/train.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train2/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


...


20 epochs completed in 0.077 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.60it/s]


                   all         50         52       0.68      0.694      0.692      0.516
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train3


lr/pg0,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg1,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▅▄▁▁▃▄▅▂▄▅▇▆▆█▇███▇█
metrics/mAP50-95(B),▅▄▁▁▃▄▄▃▄▅▆▆▇▇▇█████
metrics/precision(B),█▆▁▃▂▂▄▂▄▃▆▆▅▇██▇▇▆▆
metrics/recall(B),▁▂▅▂▆▅▅▃▄▆▇▆██▇▇████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▅▆▆▇█▇▇▇▇▆▇▆▄▄▄▃▃▂▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79720a0b1480>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
#5. using the pretrained weights of yolo medium on train1
model = YOLO("yolov8m.pt")
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs=20)

100%|██████████| 49.7M/49.7M [00:00<00:00, 173MB/s]


Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/My Drive/archive/train1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/Data/train1/labels/train.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...


20 epochs completed in 0.039 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 52.0MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.74it/s]


                   all         50         52      0.857      0.462      0.637      0.477
Speed: 0.2ms preprocess, 10.4ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train4


lr/pg0,▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
lr/pg1,▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
lr/pg2,▁▂▄▅▆▆▇▇█████▇▆▅▄▃▂▁
metrics/mAP50(B),█▂▅▅▄▃▁▂▁▁▁▁▁▁▁▁▁▄▅█
metrics/mAP50-95(B),█▁▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▃▄█
metrics/precision(B),█▂▇▅▃▃▂▂▁▁▁▁▁▁▁▁▁▃▄█
metrics/recall(B),▅▁▃▄▄▃▁▃▄▄▂▄▄▅▁▄█▇▅▅
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▂▂▁▂▃▅▇▄▇▅▆██▆▅▅▄▄▂▂


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79720a266530>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [8]:
#6. using the pretrained weights of yolo medium on train2
model = YOLO("yolov8m.pt")
model.train(data='/content/drive/My Drive/archive/train2/config.yaml', epochs=20)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/My Drive/archive/train2/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

train: Scanning /content/Data/train2/labels/train.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train2/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train7
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.100 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 52.0MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.30it/s]


                   all         50         52      0.605      0.692      0.644      0.507
Speed: 0.2ms preprocess, 9.9ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train7


lr/pg0,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg1,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▂▁▁▁▁▃▁▄▂▅▆▆▇▇▇▇████
metrics/mAP50-95(B),▁▁▁▁▁▂▁▃▂▄▅▅▆▆▆▆▇███
metrics/precision(B),▂▁▁▁▁█▁▃▂▅▅▅▆▅▅▆▅▆▅▅
metrics/recall(B),▂▄▂▂▇▁▆▅▄▃▅▅▆█▇▇████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▂▅▇██▇▇▇▆▅▅▅▄▄▄▃▃▂▂▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4b2c56d1b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

(iii) In total, you should have 6 “best” model checkpoints after the training. Train all models for the same number of epochs (e.g., 20). Report and compare the results (AP50) across the 6 variants on both the training and validation set. 

(a) How does increasing dataset size impact performance?

 (b) Does the bigger model perform better? Why or why not?

Increasing the dataset results in an increase in training accuracy always.

When trained from scratch, the validation accuracy is more when the dataset size is increased. 
When using pre-trained weights, there is not much difference in the accuracy of the model as during inference using pre trained weights, the model weights are already optimised to work robustly. 

(iv) Visualize the results of one of the above model variants on 4 images from the validation set. Pick your own diverse samples to qualitatively analyze how well the model performs.

(look at the wandb report attached)

5. [1 point] Impact of augmentations.
(i) Does the default run use augmentations? Which ones and what do they do?
Hint: analyze the output train logs (watch out for albumentations, an augmentations library).



Yes, the default run uses augmentations.

Given below are the augmentation candidates 
* Mosaic: True
* MixUp: True
* HSV: True
* HorizontalFlip: True
* AutoAugment: True
* Affine: True
* ShiftScaleRotate: True

These augmentations are implemented using the Albumentations library, which is a popular library for image augmentation.

* **Mosaic:** This augmentation combines four randomly selected images into one image by resizing and stitching them together. This helps the model learn to detect objects in different positions, scales, and contexts.

* **MixUp:** This augmentation combines two images and their corresponding bounding boxes by blending them using a random ratio. This can help the model learn better representations and improve classification performance.

* **HSV:** This augmentation randomly adjusts the Hue, Saturation, and Value (Brightness) of the input image. This helps the model become more robust to variations in lighting conditions and color shifts.

* **HorizontalFlip:** This augmentation randomly flips the input image horizontally. This helps the model learn invariance to horizontal flipping and can improve detection performance for objects that can appear in different orientations.

* **AutoAugment:** This is a collection of various image augmentations, such as sharpening, contrast adjustment, and brightness adjustment. These augmentations are applied randomly to the input images.

* **Affine:** This augmentation applies random affine transformations (translations, rotations, shears, and scales) to the input image. This helps the model learn to detect objects in different orientations and scales.

* **ShiftScaleRotate:** This augmentation applies random shifts, scales, and rotations to the input image. This also helps the model learn to detect objects in different orientations and scales.

(ii) For the yolov8n model with pretrained weights (trained on 100 images), remove all augmentations and compare performance before and after. Is the change significant? If yes, why? If not, why?

The drop down in performance is significant due to the effect of augmentations on the model. Due to the augmentations, there is a lot of variance in the training data which allows the model to learn and generalise better. Due to these augmentations, the model also becomes lees likely to overfit and retain high variance.

In [10]:
#removing all augmentations
model = YOLO("yolov8n.pt")
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs=20,
            mosaic=0.0, mixup=0.0,copy_paste=0.0, degrees=0.0, translate=0.0, scale=0.0, shear=0.0,  # NO Affine Transforms

            perspective=0.0,  # No perspective distortion
            auto_augment=None,

)


Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/archive/train1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

train: Scanning /content/Data/train1/labels/train... 279 images, 0 backgrounds, 0 corrupt: 100%|██████████| 279/279 [00:01<00:00, 166.02it/s]


train: New cache created: /content/Data/train1/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train13/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train13
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 0.047 hours.
Optimizer stripped from runs/detect/train13/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train13/weights/best.pt, 6.2MB

Validating runs/detect/train13/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.35it/s]


                   all         50         52      0.663      0.712      0.723      0.517
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/train13


lr/pg0,▂▄▅▇██▇▇▆▆▅▅▄▄▄▃▂▂▂▁
lr/pg1,▂▄▅▇██▇▇▆▆▅▅▄▄▄▃▂▂▂▁
lr/pg2,▂▄▅▇██▇▇▆▆▅▅▄▄▄▃▂▂▂▁
metrics/mAP50(B),▅▄▃▂▁▂▃▄▄▄▆▆▇▇▇▇██▇█
metrics/mAP50-95(B),▅▄▃▂▁▂▃▃▄▄▅▆▆▆▇▇▇███
metrics/precision(B),▁▁▆▆▄▅▅▅▆▆▆▇▇█▇▆▇██▇
metrics/recall(B),██▂▁▂▁▂▃▂▃▅▃▄▄▄▅▅▄▄▅
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,███▇██▇▆▆▅▅▄▄▃▃▂▂▂▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb7874532b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

(iii) Which augmentation is the most important? How did you determine this?

In [9]:
#effects of not using augmentions

#1. effect of mosaic
model = YOLO("yolov8n.pt")
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs=20,
            mosaic=0.0,  # No Mosaic
)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/archive/train2/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/Data/train2/labels/train.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train2/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


...


20 epochs completed in 0.079 hours.
Optimizer stripped from runs/detect/train8/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train8/weights/best.pt, 6.2MB

Validating runs/detect/train8/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.42it/s]


                   all         50         52      0.745      0.769      0.789      0.602
Speed: 0.4ms preprocess, 2.9ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train8


lr/pg0,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg1,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▅▆██▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▄▃▁▂▂▃▂▄▅▅▆▆▇█▇▇▇███
metrics/mAP50-95(B),▄▂▁▁▂▂▂▄▄▅▅▅▆▇▇▇▇███
metrics/precision(B),▁▄▃▄▄▄▃▅▇█▆▆▆█▇▇▇███
metrics/recall(B),█▁▁▂▃▁▂▃▂▂▅▄▅▅▅▅▅▆▅▆
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▄▅▇███▇▇▆▆▅▅▄▃▃▃▂▂▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4b2a182c80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [10]:
# 3. effect of  mixup
model = YOLO("yolov8n.pt")
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs=20,# Train YOLOv8n from pretrained weights on train1 without copy_paste

            mixup=0.0,  # No Mixup

)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/archive/train1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/Data/train1/labels/train... 174 images, 0 backgrounds, 0 corrupt: 100%|██████████| 174/174 [00:01<00:00, 91.86it/s]


train: New cache created: /content/Data/train1/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


...


20 epochs completed in 0.035 hours.
Optimizer stripped from runs/detect/train9/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train9/weights/best.pt, 6.2MB

Validating runs/detect/train9/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.44it/s]


                   all         50         52      0.717      0.712      0.695      0.491
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train9


lr/pg0,▂▃▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
lr/pg1,▂▃▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
lr/pg2,▂▃▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
metrics/mAP50(B),▇▅▅▅▃▂▃▄▃▁▂▆▆▇▇▇█▇██
metrics/mAP50-95(B),█▅▅▄▂▂▃▃▃▁▁▆▆▇▇▇█▇██
metrics/precision(B),▁▁█▆▅▆▅▅▅▅▅▆▄▇▆▆▆▆▆▇
metrics/recall(B),██▁▁▄▃▄▅▄▃▃▅▆▄▆▆▆▆▆▆
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▅▃▂▄▃▅▄▅▄▅█▄▄▄▄▃▂▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4b2ca11c30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [11]:
#4. effect of no copy paste
model = YOLO("yolov8n.pt")
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs=20,

            copy_paste=0.0,  # No Copy-Paste

)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/archive/train1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/Data/train1/labels/train.cache... 174 images, 0 backgrounds, 0 corrupt: 100%|██████████| 174/174 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


...


20 epochs completed in 0.036 hours.
Optimizer stripped from runs/detect/train10/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train10/weights/best.pt, 6.2MB

Validating runs/detect/train10/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.46it/s]


                   all         50         52      0.717      0.712      0.695      0.491
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/train10


lr/pg0,▂▃▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
lr/pg1,▂▃▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
lr/pg2,▂▃▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
metrics/mAP50(B),▇▅▅▅▃▂▃▄▃▁▂▆▆▇▇▇█▇██
metrics/mAP50-95(B),█▅▅▄▂▂▃▃▃▁▁▆▆▇▇▇█▇██
metrics/precision(B),▁▁█▆▅▆▅▅▅▅▅▆▄▇▆▆▆▆▆▇
metrics/recall(B),██▁▁▄▃▄▅▄▃▃▅▆▄▆▆▆▆▆▆
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▅▃▂▄▃▅▄▅▄▅█▄▄▄▄▃▂▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4b2d225090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [12]:
#5. effect of no affine transforms and perspective distortion

model = YOLO("yolov8n.pt")
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs=20,

            degrees=0.0, translate=0.0, scale=0.0, shear=0.0,  # NO Affine Transforms

            perspective=0.0,  # No perspective distortion

)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/archive/train1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/Data/train1/labels/train.cache... 174 images, 0 backgrounds, 0 corrupt: 100%|██████████| 174/174 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


...


20 epochs completed in 0.036 hours.
Optimizer stripped from runs/detect/train11/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train11/weights/best.pt, 6.2MB

Validating runs/detect/train11/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.11s/it]


                   all         50         52     0.0034      0.981      0.662      0.512
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to runs/detect/train11


lr/pg0,▂▃▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
lr/pg1,▂▃▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
lr/pg2,▂▃▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
metrics/mAP50(B),█▄▇▅▃▄▃▁▄▃▃▄▆▅▆▆████
metrics/mAP50-95(B),█▄▅▅▂▃▃▁▃▃▃▄▅▃▅▆▇▇▇█
metrics/precision(B),▁▁▇█▅▆▆▄▅▅▄▅▆▇▆▇▇▇▇▁
metrics/recall(B),██▃▁▃▃▅▃▅▄▅▄▅▄▅▅▅▅▅█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▆▇█▇▇▇██▇▇▅▅▄▃▂▂▂▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4b2f3608b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [11]:
#6. effect of erasing
model = YOLO("yolov8n.pt")
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs=20,

            erasing=0.0,  # No Random Erasing

)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/archive/train1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/Data/train1/labels/train.cache... 279 images, 0 backgrounds, 0 corrupt: 100%|██████████| 279/279 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


...


20 epochs completed in 0.056 hours.
Optimizer stripped from runs/detect/train14/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train14/weights/best.pt, 6.2MB

Validating runs/detect/train14/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.41it/s]


                   all         50         52      0.917      0.558      0.749       0.53
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/train14


lr/pg0,▂▄▅▇██▇▇▆▆▅▅▄▄▄▃▂▂▂▁
lr/pg1,▂▄▅▇██▇▇▆▆▅▅▄▄▄▃▂▂▂▁
lr/pg2,▂▄▅▇██▇▇▆▆▅▅▄▄▄▃▂▂▂▁
metrics/mAP50(B),▅▆▁▁▂▁▃▂▃▅▆▆▇▇▇▇▇███
metrics/mAP50-95(B),▅▆▂▁▂▁▂▂▂▅▅▆▆▇▇▇▇███
metrics/precision(B),▁▇▄▅▄▃▄▅▅▄▅▆▇▆▇▇▇▇▆█
metrics/recall(B),█▂▂▁▂▂▄▃▃▅▅▅▄▅▅▄▅▅▅▄
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▄▄▆██▇▆▆▇▆▅▆▄▄▃▃▂▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb7881e9630>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [12]:
#7. effect of auto_augment
model = YOLO("yolov8n.pt")
model.train(data='/content/drive/My Drive/archive/train1/config.yaml', epochs=20,

            auto_augment=None,  # No AutoAugment

)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/archive/train1/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/Data/train1/labels/train.cache... 279 images, 0 backgrounds, 0 corrupt: 100%|██████████| 279/279 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data/train1/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


...


20 epochs completed in 0.057 hours.
Optimizer stripped from runs/detect/train15/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train15/weights/best.pt, 6.2MB

Validating runs/detect/train15/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.59it/s]


                   all         50         52      0.917      0.558      0.749       0.53
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train15


lr/pg0,▂▄▅▇██▇▇▆▆▅▅▄▄▄▃▂▂▂▁
lr/pg1,▂▄▅▇██▇▇▆▆▅▅▄▄▄▃▂▂▂▁
lr/pg2,▂▄▅▇██▇▇▆▆▅▅▄▄▄▃▂▂▂▁
metrics/mAP50(B),▅▆▁▁▂▁▃▂▃▅▆▆▇▇▇▇▇███
metrics/mAP50-95(B),▅▆▂▁▂▁▂▂▂▅▅▆▆▇▇▇▇███
metrics/precision(B),▁▇▄▅▄▃▄▅▅▄▅▆▇▆▇▇▇▇▆█
metrics/recall(B),█▂▂▁▂▂▄▃▃▅▅▅▄▅▅▄▅▅▅▄
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▄▄▆██▇▆▆▇▆▅▆▄▄▃▃▂▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb787360a90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

#### **Observations**
As per the above results, we can see that the most important augmentation is **erasing**. It signficantly affects the loss and mAP as compared to the other augmentation parameters.The erasing augmentation randomly erases or masks out rectangular regions from the input images during training. This augmentation is designed to help the model become more robust to occlusions, where parts of the objects are obscured or hidden.

This was determined by individually removing one augmentation parameter and training the model again to see the affects in the losses, mAP and several other metrics.
